# Build a Tool-Calling Agentic AI Research Assistant with LangChain

This demo will cover building AI Agents with the legacy LangChain `AgentExecutor`. These are fine for getting started, but for working with more advanced agents and having more finer control, LangChain recommends to use LangGraph, which we cover in other courses.

Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to stop.

![](https://i.imgur.com/1uVnBAm.png)



## Install OpenAI, and LangChain dependencies

In [0]:
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14

In [0]:
!pip install markitdown

In [0]:
import os
from typing import Dict, Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, create_structured_chat_agent

## Enter Open AI API Key

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [0]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [0]:
WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

## Setup Environment Variables

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Web Search tool with information extraction
- Weather tool

![](https://i.imgur.com/TyPAYXE.png)

In [0]:
from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
import json
from warnings import filterwarnings
filterwarnings('ignore')

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
# certain websites won't let you crawl them unless you specify a user-agent
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links."""
    print('Calling web search tool')
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract content from a URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content
    # parallelize execution of different urls
    with ThreadPoolExecutor() as executor:
        for result in tqdm(results):
            try:
                future = executor.submit(extract_content, result['url'])
                # Wait for up to 15 seconds for the task to complete
                content = future.result(timeout=15)
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")

    return docs


@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather of the queried location."""
    print('Calling weather tool')
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

## Test Tool Calling with LLM

In [0]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]

chatgpt_with_tools = chatgpt.bind_tools(tools)

In [0]:
prompt = "Get details of Microsoft's earnings call Q4 2024"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

In [0]:
prompt = "how is the weather in Bangalore today"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [0]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# SYS_PROMPT = """Act as a helpful assistant.
#                 You run in a loop of Thought, Action, PAUSE, Observation.
#                 At the end of the loop, you output an Answer.
#                 Use Thought to describe your thoughts about the question you have been asked.
#                 Use Action to run one of the actions available to you - then return PAUSE.
#                 Observation will be the result of running those actions.
#                 Repeat till you get to the answer for the given user query.

#                 Use the following workflow format:
#                   Question: the input task you must solve
#                   Thought: you should always think about what to do
#                   Action: the action to take which can be any of the following:
#                             - break it into smaller steps if needed
#                             - see if you can answer the given task with your trained knowledge
#                             - call the most relevant tools at your disposal mentioned below in case you need more information
#                   Action Input: the input to the action
#                   Observation: the result of the action
#                   ... (this Thought/Action/Action Input/Observation can repeat N times)
#                   Thought: I now know the final answer
#                   Final Answer: the final answer to the original input question

#                 Tools at your disposal to perform tasks as needed:
#                   - get_weather: whenever user asks get the weather of a place.
#                   - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
#              """

# prompt_template = ChatPromptTemplate.from_messages(
#     [
#         ("system", SYS_PROMPT),
#         MessagesPlaceholder(variable_name="history", optional=True),
#         ("human", "{query}")
#     ]
# )

# prompt_template.messages

In [0]:
from langchain.agents import create_tool_calling_agent

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]

Using Chain 

In [0]:
# Create an LLM chain for deciding which tool to use
tool_selection_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a tool selection assistant. 
    Based on the user's question, determine which tool would be most appropriate.
    Respond with only the tool name: "search_web_extract_info", "get_weather" .
    If none of these tools can help, respond with "None"."""),
    ("human", "{query}")
])

tool_selection_chain = LLMChain(
    llm=chatgpt,
    prompt=tool_selection_prompt,
    output_key="selected_tool"
)

In [0]:
# Create an LLM chain for formatting tool inputs
tool_input_prompt = ChatPromptTemplate.from_messages([
    ("system", """Extract the specific input needed for the tool from the user's query.
    For WeatherService: Extract the location name.
    for get_weather extract the keywords and search the info in web
    Provide only the extracted information, nothing else."""),
    ("human", "Tool to use: {selected_tool}\nUser query: {query}")
])

tool_input_chain = LLMChain(
    llm=chatgpt,
    prompt=tool_input_prompt,
    output_key="tool_input"
)

In [0]:
from langchain_core.output_parsers import StrOutputParser

In [0]:
# Create an LLM chain for formatting the final response
response_prompt = ChatPromptTemplate.from_messages([
    ("system", "Create a helpful response to the user's query using the tool's output."),
    ("human", "User query: {query}\nTool used: {selected_tool}\nTool output: {tool_output}")
])

response_chain = LLMChain(
    llm=chatgpt,
    prompt=response_prompt,
    output_key="response",
    output_parser=StrOutputParser()
)

In [0]:
# Function to run the full chain
def process_query(query: str) -> str:
    # First, select which tool to use
    tool_selection_result = tool_selection_chain.invoke({"query": query})
    selected_tool = tool_selection_result["selected_tool"].strip()
    print(f"Selected tool: {selected_tool}")
    
    if selected_tool == "None":
        return "I don't have a tool that can help with this query."
    
    # Next, format the input for the selected tool
    tool_input_result = tool_input_chain.invoke({
        "selected_tool": selected_tool,
        "query": query
    })
    tool_input = tool_input_result["tool_input"].strip()
    # print(f"Tool input: {tool_input}")
    
    # Find and run the appropriate tool
    tool_output = "Tool not found"
    for tool in tools:
        if tool.name == selected_tool:
            tool_output = tool.func(tool_input)
            break
    # print(f"Tool output: {tool_output}")
    
    # Format the final response
    response_result = response_chain.invoke({
        "query": query,
        "selected_tool": selected_tool,
        "tool_output": tool_output
    })
    return response_result["response"]

In [0]:
examples = [
        "What is 234 * 78.5?",
        "What's the weather like in San Francisco?",
        "Tell me about LangChain.",
    ]
    
for example in examples:
    print(f"\n\nQUERY: {example}")
    print("-" * 50)
    response = process_query(example)
    print("FINAL RESPONSE:")
    print(response)